# 0. 로그인한 유저의 장르 매트릭스를 만듦( 얘가 뭘 주로 보는 앤가 파악)

# 1. 네이버 뉴스검색 api사용해서 키워드 검색
# 2. 긁어온 뉴스에 대해 만든 엔진써서 장르분류
# 3. 장르 매트릭스에 따라 자주 보는 장르를 위로 올림
(predict or predict_proba, 확률기준이 더 복잡함)

1. 분류됬을 때 확률 고만고만한 애들은 뒤로 보냄
2. ~~user_pref에 따라 추천할 때 확률이 높은 애들부터 추천~~

path = [('정치',0), ('경제',1), ('사회',2), ('세계',3), ('생활문화',4), ('IT과학',5)]

In [1]:
import MySQLdb as db
import pandas as pd
import numpy as np
from naver_news_search import api_search

In [2]:
import json

with open('/home/ej/github/news_config.json') as f:
    config = json.load(f)

db = db.connect(
    config['ADDR'],
    config['ID'],
    config['PW'],
    config['DB'],
    charset='utf8'
)

In [3]:
view = pd.read_sql('SELECT * FROM VIEW', db)
view['count'] = 1
article = pd.read_sql('SELECT * FROM article', db)

merged = pd.merge(view, article,left_on='article_id', right_on='article_id')
merged = merged.loc[:,['user_id','article_id','area','count']]

genre_matrix = merged.pivot_table(values='count',index='user_id',columns='area',aggfunc=np.sum)

In [4]:
genre_matrix.head()

area,0,1,2,3,4,5
user_id,,,,,,
1,13,11,6,12,11,11
2,9,10,12,6,7,11
3,7,7,13,9,14,14
4,17,10,13,10,12,15
5,11,19,12,11,14,11


In [5]:
data = api_search('파이썬',10)['items']

In [6]:
crawling_list = []
for each_news in data:
    crawling_list.append(each_news['link'])

In [7]:
def go_get_article(crawling_list):
    import pandas as pd
    import time
    from bs4 import BeautifulSoup
    import requests

    article_list = []

    for idx, val in enumerate(crawling_list):
        html = val
        response = requests.get(html)
        web_elem = BeautifulSoup(response.content, 'html.parser')
        p_tags = web_elem.find_all('p')

        article = ''

        for each in p_tags:
            article += each.text

        article_list.append(article)

        if idx % 10 == 0:
            print('{} completed, {} total'.format(idx + 1, len(crawling_list)))

    return article_list

In [8]:
tmp = go_get_article(crawling_list)

1 completed, 10 total


In [9]:
tmp = [each.split('.') for each in tmp]

In [10]:
def trimming(articles):
    import re

    is_email = []
    for each in articles:
        tmp = re.findall('[^@]+@[^@]+\.[^@]+', each)
        is_email.append(tmp)

    is_email = [True if len(each) != 0 else False for each in is_email]
    upper_bound = len(
        is_email) - 2 - is_email[::-1].index(True) if True in is_email else len(is_email) - 1

    articles = [articles[idx] if not is_email[idx]
                else '' for idx in range(0, upper_bound)]

    # 2.
    for idx, val in enumerate(articles):
        converted = re.sub('[^가-힣0-9a-zA-Z.\\s]', ' ', val)
        articles[idx] = converted

    # 3.
    articles = [each for each in articles if each != '']

    return articles

In [11]:
tmp = [''.join(trimming(each)) for each in tmp]

In [3]:
import pickle
from konlpy.tag import Twitter

In [5]:
pos_tagger = Twitter()
def tokenize_pos(doc):
    return ["/".join(t) for t in pos_tagger.pos(doc)]
model = pickle.load(open('dataset/twitter-tfidf-mulnb.pkl','rb'))

# 확률이 전부 고만고만한 애들은 제외

In [13]:
news_content = pd.DataFrame(columns=['content'])
news_content['content'] = pd.Series(tmp)

In [14]:
news_content.head()

,content
0,4 ...
1,\n해당 언론사가 채널 주요 뉴스로직접 선정한 기사입니다\n채널 안내\n닫기\n설정...
2,사용중인 브라우저의 쿠키가 차단되어 있을 경우 백서를 다운로드 받을 수 없습니다 브...
3,2 30여 년째 주인 찾지 못한 안면도 개발 사업 충남일보 최솔 기자 1조 원대 ...
4,


In [46]:
X_test = pd.Series(tmp)

In [47]:
%%time
y_pred = model.predict(X_test)

Wall time: 260 ms


In [48]:
%%time
y_prob = model.predict_proba(X_test)

Wall time: 201 ms


In [18]:
y_pred

array([5, 4, 5, 1, 5, 4, 4, 5, 5, 5], dtype=int64)

In [20]:
# 키워드가 파이썬인데 사회로 분류된 애들
y_pred.argsort()[:5]

array([3, 1, 5, 6, 0], dtype=int64)

In [21]:
genre_matrix.loc[1]

area
0    13
1    11
2     6
3    12
4    11
5    11
Name: 1, dtype: int64

In [22]:
pd.DataFrame(y_prob).head()

,0,1,2,3,4,5
0,0.003900,0.155258,0.020840,0.161781,0.239972,0.418248
1,0.014827,0.128224,0.213606,0.008843,0.459370,0.175130
2,0.003605,0.002916,0.011103,0.006945,0.008815,0.966616
3,0.000111,0.993156,0.003275,0.000246,0.000800,0.002411
4,0.168211,0.158232,0.169874,0.165835,0.167261,0.170587


In [23]:
pd.DataFrame(y_prob).sort_values(by=[0], axis=0, ascending=False).index

Int64Index([4, 1, 5, 6, 0, 2, 9, 3, 8, 7], dtype='int64')

In [24]:
pd.DataFrame(y_prob).sort_values(by=[0], axis=0, ascending=False)

,0,1,2,3,4,5
4,0.168211,0.158232,0.169874,0.165835,0.167261,0.170587
1,0.014827,0.128224,0.213606,0.008843,0.459370,0.175130
5,0.014827,0.128224,0.213606,0.008843,0.459370,0.175130
6,0.014827,0.128224,0.213606,0.008843,0.459370,0.175130
0,0.003900,0.155258,0.020840,0.161781,0.239972,0.418248
2,0.003605,0.002916,0.011103,0.006945,0.008815,0.966616
9,0.000406,0.265882,0.096839,0.003141,0.008809,0.624924
3,0.000111,0.993156,0.003275,0.000246,0.000800,0.002411
8,0.000025,0.252990,0.014094,0.002444,0.006768,0.723680
7,0.000017,0.005721,0.009411,0.000148,0.008772,0.975930


In [41]:
def news_rank(user_id, genre_matrix, y_pred, y_prob):
    user_pref = genre_matrix.loc[user_id].sort_values(ascending=False).index

    print(user_pref, '\n')

    article = []

    for area in range(0, 6):
        each_area = np.argwhere(y_pred == area)
        each_area = each_area.reshape(1, len(each_area))
        article.append(each_area)

    prob_df = pd.DataFrame(y_prob)
    sorted_article = []

    print('sorted by probability')

    for area, each in enumerate(article):
        sorter = prob_df.loc[each[0], :]
        sorter.sort_values(by=area, axis=0, inplace=True, ascending=False)
        sorted_article.append(list(sorter.index))

    for area in user_pref:
        print(area, sorted_article[area])

    news_rank = []
    for pref_area in user_pref:
        for each_article in sorted_article[pref_area]:
            news_rank.append(each_article)

    return news_rank

In [44]:
ranked_news = news_rank(1,genre_matrix, y_pred,y_prob)

Int64Index([0, 3, 5, 4, 1, 2], dtype='int64', name='area') 

sorted by probability
0 []
3 []
5 [7, 2, 8, 9, 0, 4]
4 [1, 5, 6]
1 [3]
2 []


In [45]:
# 뉴스랭크에 따른 검색 추천 리스트
for each in ranked_news:
    print(crawling_list[each])

http://www.getnews.co.kr/news/articleView.html?idxno=62918
http://www.itworld.co.kr/techlibrary/108969
http://www.metroseoul.co.kr/news/newsview?newscd=2018041500072
http://edu.donga.com/?p=article&ps=view&at_no=20180413175338210274
http://www.readersnews.com/news/articleView.html?idxno=79191
http://www.thebell.co.kr/front/free/contents/news/article_view.asp?key=201804170100033220002051
http://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=101&oid=030&aid=0002700203
http://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=105&oid=215&aid=0000623309
http://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=102&oid=020&aid=0003141011
http://www.chungnamilbo.com/news/articleView.html?idxno=444192


In [2]:
from news_ranker import *

In [3]:
news_rank_recommend(user_id = 1,keyword = '파이썬',how_many = 10)

1 completed, 10 total
1 's preference list =  Int64Index([0, 3, 5, 4, 1, 2], dtype='int64', name='area') 

sorted by probability
0 [4]
3 []
5 [7, 2, 8, 9, 0]
4 [1, 5, 6]
1 [3]
2 []


['http://www.thebell.co.kr/front/free/contents/news/article_view.asp?key=201804170100033220002051',
 'http://www.getnews.co.kr/news/articleView.html?idxno=62918',
 'http://www.itworld.co.kr/techlibrary/108969',
 'http://www.metroseoul.co.kr/news/newsview?newscd=2018041500072',
 'http://edu.donga.com/?p=article&ps=view&at_no=20180413175338210274',
 'http://www.readersnews.com/news/articleView.html?idxno=79191',
 'http://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=101&oid=030&aid=0002700203',
 'http://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=105&oid=215&aid=0000623309',
 'http://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=102&oid=020&aid=0003141011',
 'http://www.chungnamilbo.com/news/articleView.html?idxno=444192']